# Sabati

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#1. Caricamento dati
delivery_data = pd.read_csv('delivery_history.csv').copy()

#2. conversione variabile delivery_date in datetime
delivery_data['delivery_date'] = pd.to_datetime(delivery_data['delivery_date'], errors='coerce')
# Crea una seconda colonna solo con la data, tipo object (utile solo per visualizzazioni)
delivery_data['delivery_day'] = delivery_data['delivery_date'].dt.date

#3. Creazione di colonne derivate dalla data
#Giorno della settimana: 0=Monday, 6=Sunday
delivery_data['weekday_num'] = delivery_data['delivery_date'].dt.dayofweek

#4. Divisione delle singole variabili temporali
# Mese (1-12)
delivery_data['month'] = delivery_data['delivery_date'].dt.month
# Anno
delivery_data['year'] = delivery_data['delivery_date'].dt.year
# Periodo (Anno-Mese) utile per analisi mensili
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

#5. Conversione is_event in booleano
valid_bool_mask = delivery_data['is_event'].isin([0, 1, True, False])
# Applica conversione solo ai validi (opzionale: puoi forzare tutto a bool, ma questo è più sicuro)
delivery_data.loc[valid_bool_mask, 'is_event'] = delivery_data.loc[valid_bool_mask, 'is_event'].astype(bool)
delivery_data = delivery_data[delivery_data['is_event'] == True].copy()

#6. filtraggio dati solo per sabato
saturday_data = delivery_data[delivery_data['weekday_num'] == 5].copy()
saturday_data.reset_index(drop=True, inplace=True)
print(delivery_data.columns)

#7. Analisi esplorativa
# Statistiche descrittive
print(saturday_data.describe(include='all'))
# Conteggio eventi per mese
event_count = saturday_data['year_month'].value_counts().sort_index()
print(event_count)  
# Visualizzazione distribuzione eventi per mese
plt.figure(figsize=(12, 6)) 
sns.barplot(x=event_count.index.astype(str), y=event_count.values, palette='viridis')
plt.xticks(rotation=45)
plt.title('Numero di Eventi di Consegna per Mese (Solo Sabato)')  
for i, v in enumerate(event_count.values):
    plt.text(i, v + 0.5, str(v), ha='center', va='bottom')  
plt.xlabel('Mese')
plt.ylabel('Numero di Eventi')
plt.tight_layout()
plt.show()

print(delivery_data)



In [ ]:
# Create a summary table for clients who order on Saturdays
saturday_customers_ids = saturday_data['location_id'].unique()
summary_table = pd.DataFrame({
    'location_id': saturday_customers_ids,
    'total_quantity': [delivery_data[delivery_data['location_id'] == id]['quantity'].sum() for id in saturday_customers_ids],
    'saturday_quantity': [saturday_data[saturday_data['location_id'] == id]['quantity'].sum() for id in saturday_customers_ids],
    'saturday_orders': [saturday_data[saturday_data['location_id'] == id].shape[0] for id in saturday_customers_ids]
})

# Sort by total quantity descending
summary_table = summary_table.sort_values('total_quantity', ascending=False)

# Calculate percentages
summary_table['saturday_percent'] = (summary_table['saturday_quantity'] / summary_table['total_quantity'] * 100).round(1)

# Format the table for display
pd.set_option('display.float_format', lambda x: '%.1f' % x)
print(summary_table[summary_table['saturday_percent']>10])

In [ ]:
#clienti che ordinano almeno una volta il sabato (count)
saturday_customers = saturday_data['location_id'].nunique()
print(f"Numero di clienti che hanno ordinato almeno una volta il sabato: {saturday_customers}")

In [ ]:
# Filtra i clienti con saturday_percent > 10
filtered_summary = summary_table[summary_table['saturday_percent'] > 10].copy()

# Preleva da delivery_data la lat e lon solo per i clienti filtrati
coords = delivery_data[['location_id', 'lat', 'lon']].drop_duplicates(subset='location_id')

# Unisci lat e lon a filtered_summary tramite location_id
final_summary = filtered_summary.merge(coords, on='location_id', how='left')

# Visualizza
print(final_summary)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.scatter(final_summary['lon'], final_summary['lat'], color='blue')

for _, row in final_summary.iterrows():
    plt.text(
        row['lon'], row['lat'],
        f"{int(row['location_id'])} ({int(row['total_quantity'])};{int(row['saturday_quantity'])};" +
        r"$\mathbf{" + f"{int(round(row['saturday_percent']))}" + r"}$%)",
        fontsize=8, verticalalignment='bottom', horizontalalignment='right', color='black'
    )

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title('Clienti con percentuale sabato > 10%')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

# Step 1: trova i clienti che hanno almeno un ordine di sabato
clients_with_saturday = delivery_data[delivery_data['weekday_num'] == 5]['location_id'].unique()

# Step 2: filtra il dataset per questi clienti
filtered_data = delivery_data[delivery_data['location_id'].isin(clients_with_saturday)].copy()

# Step 3: mappa weekday_num a nomi giorni settimana
weekday_map = {0: 'Lunedì', 1: 'Martedì', 2: 'Mercoledì', 3: 'Giovedì', 4: 'Venerdì', 5: 'Sabato', 6: 'Domenica'}
filtered_data['weekday_name'] = filtered_data['weekday_num'].map(weekday_map)

# Step 4: calcola numero di giorni distinti in cui il cliente ordina per giorno della settimana
days_ordered_per_weekday = filtered_data.groupby(['location_id', 'weekday_name'])['delivery_day'].nunique().reset_index(name='num_days_ordered')

# Step 5: calcola quantità totale per location_id e weekday
quantity_per_location_weekday = filtered_data.groupby(['location_id', 'weekday_name'])['quantity'].sum().reset_index(name='total_quantity')

# Step 6: calcola quantità totale per cliente (tutti i giorni)
total_quantity_per_client = filtered_data.groupby('location_id')['quantity'].sum().reset_index(name='total_quantity_client')

# Step 7: calcola quantità ordinata solo il sabato per cliente
saturday_data = filtered_data[filtered_data['weekday_num'] == 5]
saturday_quantity_per_client = saturday_data.groupby('location_id')['quantity'].sum().reset_index(name='saturday_quantity')

# Step 8: unisci quantità per giorno con quantità totale cliente e quantità sabato
quantity_df = quantity_per_location_weekday.merge(total_quantity_per_client, on='location_id', how='left')
quantity_df = quantity_df.merge(saturday_quantity_per_client, on='location_id', how='left').fillna(0)

# Step 9: calcola rapporto sabato / totale cliente
quantity_df['saturday_ratio'] = (quantity_df['saturday_quantity'] / quantity_df['total_quantity_client']).round(3)

# Step 10: unisci con giorni ordinati
final_summary = days_ordered_per_weekday.merge(quantity_df, on=['location_id', 'weekday_name'], how='left')

# Step 11: filtra e stampa righe con rapporto sabato > 0.1
print(final_summary[final_summary['saturday_ratio'] > 0.1][['location_id', 'weekday_name', 'num_days_ordered', 'total_quantity', 'saturday_quantity', 'saturday_ratio']])



In [ ]:
import numpy as np

# --- Heatmap quantità ordinata, senza annotazione per zeri ---
quantity_annot = quantity.astype(str)
quantity_annot[quantity == 0] = ''

plt.figure(figsize=(20, 25))
sns.heatmap(
    quantity, 
    cmap=sns.light_palette("blue", as_cmap=True), 
    annot=quantity_annot, 
    fmt="s",
    cbar_kws={'label': 'Quantità ordinata'}
)
plt.title('Quantità ordinata per cliente e giorno della settimana')
plt.xlabel('Giorno della settimana')
plt.ylabel('Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Heatmap numero giorni ordinati, senza annotazione per zeri ---
days_annot = days_ordered_pivot.astype(str)
days_annot[days_ordered_pivot == 0] = ''

plt.figure(figsize=(20, 25))
sns.heatmap(
    days_ordered_pivot, 
    cmap=sns.light_palette("blue", as_cmap=True), 
    annot=days_annot, 
    fmt="s",
    cbar_kws={'label': 'Numero giorni ordinati'}
)
plt.title('Numero di giorni con ordini per cliente e giorno della settimana')
plt.xlabel('Giorno della settimana')
plt.ylabel('Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

# Crea annotazioni senza zeri per quantità
quantity_annot = quantity_pivot_filtered.astype(str)
quantity_annot[quantity_pivot_filtered == 0] = ''

plt.figure(figsize=(10, 4))
sns.heatmap(
    quantity_pivot_filtered,
    cmap=sns.light_palette("blue", as_cmap=True),
    annot=quantity_annot,
    fmt="s",
    cbar_kws={'label': 'Quantità ordinata'}
)
plt.title('Quantità ordinata per cliente (saturday_ratio > 0.1) e giorno della settimana')
plt.xlabel('Giorno della settimana')
plt.ylabel('Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Crea annotazioni senza zeri per giorni ordinati
days_annot = days_ordered_pivot_filtered.astype(str)
days_annot[days_ordered_pivot_filtered == 0] = ''

plt.figure(figsize=(10, 4))
sns.heatmap(
    days_ordered_pivot_filtered,
    cmap=sns.light_palette("blue", as_cmap=True),
    annot=days_annot,
    fmt="s",
    cbar_kws={'label': 'Numero giorni ordinati'}
)
plt.title('Numero di giorni con ordini per cliente (saturday_ratio > 0.1) e giorno della settimana')
plt.xlabel('Giorno della settimana')
plt.ylabel('Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Analisi ABC 

In [ ]:
import matplotlib.pyplot as plt

# Calcola numero totale ordini per cliente
orders_per_client = delivery_data.groupby('location_id').size().reset_index(name='num_orders')

# Calcola quantità totale all'anno per cliente (somma quantity)
quantity_per_client = delivery_data.groupby('location_id')['quantity'].sum().reset_index(name='quantity_total')

# Ordina per numero ordini decrescente e prendi top 40
top_clients = orders_per_client.sort_values('num_orders', ascending=False).head(40)

# Mantieni solo clienti top anche in quantity
top_quantity = quantity_per_client[quantity_per_client['location_id'].isin(top_clients['location_id'])]

# Ordina quantity in base a ordini per allineare
top_quantity = top_quantity.set_index('location_id').loc[top_clients['location_id']].reset_index()
# Plot
fig, ax1 = plt.subplots(figsize=(15, 8))

# Barre numero ordini celesti
bars = ax1.bar(top_clients['location_id'].astype(str), top_clients['num_orders'], label='Numero ordini', color='skyblue')
ax1.set_ylabel('Numero di ordini', color='skyblue')
ax1.set_xlabel('Cliente (location_id)')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticklabels(top_clients['location_id'].astype(str), rotation=90)

# Aggiungi etichette sopra le barre
for bar in bars:
    height = bar.get_height()
    ax1.annotate(f'{int(height)}', xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0,3), textcoords='offset points',
                 ha='center', va='bottom', fontsize=9, color='skyblue')

# Asse y destro per quantità totale
ax2 = ax1.twinx()
ax2.set_ylabel('Quantità totale ordinata', color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Punti quantità neri
ax2.plot(top_quantity['location_id'].astype(str), top_quantity['quantity_total'], 'ko', label='Quantità totale')

# Aggiungi etichette con quantità accanto ai punti neri
for i, row in top_quantity.iterrows():
    ax2.annotate(f"{int(row['quantity_total'])}", (str(row['location_id']), row['quantity_total']),
                 textcoords="offset points", xytext=(5,0), ha='left', color='black', fontsize=9)

# Legenda unificata
fig.legend(loc='upper right', bbox_to_anchor=(0.85, 0.9))

plt.title('Top 40 clienti per numero ordini con quantità totale ordinata')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Prendi i primi 40 clienti da top_clients ordinati per numero ordini
top_40_clients = top_clients.sort_values('num_orders', ascending=False).head(25)

# Estrarre lat e lon unici da delivery_data per tutti i clienti top 40
coords = delivery_data[delivery_data['location_id'].isin(top_40_clients['location_id'])][['location_id', 'lat', 'lon']].drop_duplicates('location_id')

# Unisci top_clients con le coordinate
top_40_data = top_40_clients.merge(coords, on='location_id', how='left')

plt.figure(figsize=(30,8))
plt.scatter(top_40_data['lon'], top_40_data['lat'], color='green')

for _, row in top_40_data.iterrows():
    plt.text(
        row['lon'], row['lat'], 
        f"{int(row['location_id'])} ({int(row['num_orders'])})", 
        fontsize=9, verticalalignment='bottom', horizontalalignment='right', color='black'
    )

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title('Mappa Top 25 clienti con numero ordini (tra parentesi)')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calcola quantità totale ordinata per cliente
quantity_per_client = delivery_data.groupby('location_id')['quantity'].sum().reset_index(name='quantity_total')
print(quantity_per_client.columns)
# Calcola numero totale ordini per cliente
orders_per_client = delivery_data.groupby('location_id').size().reset_index(name='num_orders')

# Ordina per quantità totale decrescente e prendi top 40
top_quantity_clients = quantity_per_client.sort_values('quantity_total', ascending=False).head(40)

# Mantieni solo clienti top anche nel conteggio ordini
top_orders = orders_per_client[orders_per_client['location_id'].isin(top_quantity_clients['location_id'])]

# Ordina ordini per corrispondere all'ordinamento quantità
top_orders = top_orders.set_index('location_id').loc[top_quantity_clients['location_id']].reset_index()

# Plot
fig, ax1 = plt.subplots(figsize=(25, 8))

# Barre quantità totale celesti
bars = ax1.bar(top_quantity_clients['location_id'].astype(str), top_quantity_clients['quantity_total'], label='Quantità totale', color='skyblue')
ax1.set_ylabel('Quantità totale ordinata', color='skyblue')
ax1.set_xlabel('Cliente (location_id)')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticklabels(top_quantity_clients['location_id'].astype(str), rotation=90)

# Aggiungi etichette sopra le barre
for bar in bars:
    height = bar.get_height()
    ax1.annotate(f'{int(height)}', xy=(bar.get_x() + bar.get_width() / 2, height),
                 xytext=(0,3), textcoords='offset points',
                 ha='center', va='bottom', fontsize=9, color='skyblue')

# Asse y destro per numero ordini
ax2 = ax1.twinx()
ax2.set_ylabel('Numero di ordini', color='black')
ax2.tick_params(axis='y', labelcolor='black')

# Punti neri per numero ordini
ax2.plot(top_orders['location_id'].astype(str), top_orders['num_orders'], 'ko', label='Numero ordini')

# Aggiungi etichette numero ordini accanto a punti neri
for i, row in top_orders.iterrows():
    ax2.annotate(f"{int(row['num_orders'])}", (str(row['location_id']), row['num_orders']),
                 textcoords="offset points", xytext=(5,0), ha='left', color='black', fontsize=9)

# Legenda unificata
fig.legend(loc='upper right', bbox_to_anchor=(0.85, 0.9))

plt.title('Top 40 clienti per quantità ordinata con numero di ordini')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Prendi i top 40 clienti da quantity_per_client (già calcolato)
top_40_clients = quantity_per_client.sort_values('quantity_total', ascending=False).head(25)

# Preleva lat e lon unici da delivery_data per clienti top 40
coords = delivery_data[delivery_data['location_id'].isin(top_40_clients['location_id'])][['location_id', 'lat', 'lon']].drop_duplicates('location_id')

# Unisci coords a top_40_clients
top_40_clients = top_40_clients.merge(coords, on='location_id', how='left')

plt.figure(figsize=(12,8))
plt.scatter(top_40_clients['lon'], top_40_clients['lat'], color='green')

for _, row in top_40_clients.iterrows():
    plt.text(
        row['lon'], row['lat'],
        f"{int(row['location_id'])} ({int(row['quantity_total'])})",
        fontsize=9, verticalalignment='bottom', horizontalalignment='right', color='black'
    )

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title('Mappa Top 25 clienti per quantità totale ordinata')
plt.grid(True)
plt.show()


In [ ]:
# Calculate total quantity per client
quantity_per_client = delivery_data.groupby('location_id')['quantity'].sum().reset_index(name='total_quantity')

# Sort in descending order
quantity_per_client = quantity_per_client.sort_values(by='total_quantity', ascending=False)

# Calculate percentage of total
total_quantity_sum = quantity_per_client['total_quantity'].sum()
quantity_per_client['percentage'] = (quantity_per_client['total_quantity'] / total_quantity_sum * 100).round(2)

# Calculate cumulative percentage
quantity_per_client['cumulative_percentage'] = quantity_per_client['percentage'].cumsum().round(2)

print(quantity_per_client)

num_clients_80 = quantity_per_client[quantity_per_client['cumulative_percentage'] <= 80].shape[0]
print(f"Numero clienti con cumulata fino all'80%: {num_clients_80}")

# Numero totale clienti
total_clients = quantity_per_client.shape[0]

# Numero clienti fino all'80% cumulato
num_clients_80 = quantity_per_client[quantity_per_client['cumulative_percentage'] <= 80].shape[0]

# Calcola percentuale
percent_clients_80 = (num_clients_80 / total_clients) * 100

print(f"Percentuale clienti che coprono l'80% della quantità totale: {percent_clients_80:.2f}%")



In [ ]:
# Calcola numero totale ordini per cliente
orders_per_client = delivery_data.groupby('location_id').size().reset_index(name='total_orders')

# Ordina in ordine decrescente
orders_per_client = orders_per_client.sort_values(by='total_orders', ascending=False)

# Calcola percentuale di ordini rispetto al totale complessivo
total_orders_sum = orders_per_client['total_orders'].sum()
orders_per_client['percentage'] = (orders_per_client['total_orders'] / total_orders_sum * 100).round(2)

# Calcola la percentuale cumulata
orders_per_client['cumulative_percentage'] = orders_per_client['percentage'].cumsum().round(2)

print(orders_per_client)

# Numero clienti che coprono cumulata fino all'80%
num_clients_80_orders = orders_per_client[orders_per_client['cumulative_percentage'] <= 80].shape[0]
print(f"Numero clienti con cumulata ordini fino all'80%: {num_clients_80_orders}")

# Numero totale clienti
total_clients_orders = orders_per_client.shape[0]

# Calcola percentuale clienti che coprono l'80% ordini
percent_clients_80_orders = (num_clients_80_orders / total_clients_orders) * 100
print(f"Percentuale clienti che coprono l'80% degli ordini: {percent_clients_80_orders:.2f}%")


In [ ]:
# Numero totale di ordini
total_orders = delivery_data.shape[0]

# Numero di clienti unici
num_clients = delivery_data['location_id'].nunique()

# Statistiche quantità (media, mediana, minimo, massimo)
quantity_stats = delivery_data['quantity'].describe()

# Numero di ordini per cliente (senza visualizzare tutti)
orders_per_client = delivery_data.groupby('location_id').size()

# Statistiche di ordini per cliente (media, mediana, max)
orders_stats = orders_per_client.describe()

# Numero di ordini validi (dove is_event == 1)
valid_orders = delivery_data[delivery_data['is_event'] == 1]
num_valid_orders = valid_orders.shape[0]

# Quantità totale e media quantità per ordine valido
total_quantity_valid = valid_orders['quantity'].sum()
mean_quantity_valid = valid_orders['quantity'].mean()

print(f"Totale ordini: {total_orders}")
print(f"Clienti unici: {num_clients}")
print(f"Statistiche quantità:\n{quantity_stats}")
print(f"Statistiche ordini per cliente:\n{orders_stats}")
print(f"Totale ordini validi (is_event=1): {num_valid_orders}")
print(f"Quantità totale ordini validi: {total_quantity_valid}")
print(f"Media quantità per ordine valido: {mean_quantity_valid:.2f}")


# stampare i due punti con lat e long molto alti

In [ ]:
# Assumiamo che delivery_data sia un DataFrame con almeno le colonne:
# location_id, lon, quantity, weekday_num

# Filtra clienti con lon fuori dal range [5, 12.5]
out_of_range_clients = delivery_data[(delivery_data['lon'] < 5) | (delivery_data['lon'] > 12.5)]['location_id'].unique()

# Prendi i dati solo per questi clienti
filtered_data = delivery_data[delivery_data['location_id'].isin(out_of_range_clients)]

# Calcola quantità totale ordinata e numero totale ordini per cliente
agg_total = filtered_data.groupby('location_id').agg(
    total_quantity=('quantity', 'sum'),
    total_orders=('quantity', 'size')
).reset_index()

# Calcola quantità e numero ordini effettuati il sabato (weekday_num == 5) per cliente
data_saturday = filtered_data[filtered_data['weekday_num'] == 5]
agg_saturday = data_saturday.groupby('location_id').agg(
    saturday_quantity=('quantity', 'sum'),
    saturday_orders=('quantity', 'size')
).reset_index()

# Calcola varianza, deviazione standard e coefficiente di variazione sulla quantità per cliente
agg_stats = filtered_data.groupby('location_id')['quantity'].agg(['var', 'std', 'mean']).reset_index()
agg_stats['cv'] = agg_stats['std'] / agg_stats['mean']

# Unisci tutti i dati
result = agg_total.merge(agg_saturday, on='location_id', how='left').merge(agg_stats, on='location_id', how='left')

# Riempie NaN per clienti senza ordini di sabato
result[['saturday_quantity', 'saturday_orders']] = result[['saturday_quantity', 'saturday_orders']].fillna(0)

# Arrotonda valori statistici a due decimali
result[['var', 'std', 'cv','mean']] = result[['var', 'std', 'cv','mean']].round(2)

print(result)


In [ ]:
mean_colli_per_order = delivery_data['quantity'].mean()
var_colli_per_order = delivery_data['quantity'].var()
std_colli_per_order = delivery_data['quantity'].std()
cv_colli_per_order = std_colli_per_order / mean_colli_per_order

print(f"Media di colli per singolo ordine (tutti clienti): {mean_colli_per_order:.2f}")
print(f"Varianza di colli per singolo ordine (tutti clienti): {var_colli_per_order:.2f}")
print(f"Coefficiente di variazione di colli per singolo ordine (tutti clienti): {cv_colli_per_order:.2f}")


In [ ]:
# Quantità totale ordinata su tutti i clienti
total_quantity_all = delivery_data['quantity'].sum()

# Numero clienti distinti in tutto il dataset
num_clients_all = delivery_data['location_id'].nunique()

# Media volumi per cliente su tutto il dataset
mean_volume_per_client_all = total_quantity_all / num_clients_all

print(f"Quantità totale ordinata (tutti i clienti): {total_quantity_all}")
print(f"Numero di clienti distinti (tutti i clienti): {num_clients_all}")
print(f"Media totale di volumi per cliente (tutti i clienti): {mean_volume_per_client_all:.2f}")


In [ ]:
client_ids = [14930, 15133]

columns_to_show = ['location_id', 'quantity', 'delivery_date', 'delivery_day', 'weekday_num', 'month', 'year', 'year_month']

filtered_data = delivery_data[
    (delivery_data['location_id'].isin(client_ids)) & (delivery_data['is_event'] == 1)
][columns_to_show]

print(filtered_data)


# Come varia la domanda tra i clienti

In [ ]:
import pandas as pd

# Prepara il dato con aggregazione mensile per cliente
monthly_demand = delivery_data.groupby(['location_id', delivery_data['delivery_date'].dt.to_period('M')])['quantity'].sum().reset_index()
monthly_demand.rename(columns={'delivery_date': 'year_month'}, inplace=True)

# Calcola indicatori di variabilità della domanda per cliente
stats_cliente = monthly_demand.groupby('location_id')['quantity'].agg(
    mean_monthly_demand='mean',
    std_monthly_demand='std',
    min_monthly_demand='min',
    max_monthly_demand='max'
).reset_index()

# Calcola coefficiente di variazione (CV)
stats_cliente['cv_monthly_demand'] = stats_cliente['std_monthly_demand'] / stats_cliente['mean_monthly_demand']

# Calcola rapporto min/max domanda mensile
stats_cliente['min_max_ratio'] = stats_cliente['min_monthly_demand'] / stats_cliente['max_monthly_demand']

print(stats_cliente.head())


In [ ]:
# Conta clienti con cv > 0.5
num_clients_high_cv = (stats_cliente['cv_monthly_demand'] > 0.5).sum()

print(f"Numero di clienti con coefficiente di variazione maggiore di 0.5: {num_clients_high_cv}")


In [ ]:
import matplotlib.pyplot as plt

# Calcola la quantità totale per cliente
total_quantity_per_client = delivery_data.groupby('location_id')['quantity'].sum().reset_index()
total_quantity_per_client.rename(columns={'quantity': 'total_quantity'}, inplace=True)

# Trova i top 20 clienti per quantità totale ordinata
top_20_total_qty = total_quantity_per_client.sort_values('total_quantity', ascending=False).head(20)

# Calcola la quantità mensile per cliente
delivery_data['year_month'] = delivery_data['delivery_date'].dt.to_period('M')

monthly_quantity = delivery_data.groupby(['location_id', 'year_month'])['quantity'].sum().reset_index()

# Calcola la media mensile e la deviazione standard mensile per i clienti top 20
stats_monthly = monthly_quantity[monthly_quantity['location_id'].isin(top_20_total_qty['location_id'])].groupby('location_id')['quantity'].agg(['mean', 'std']).reset_index()
stats_monthly.rename(columns={'mean': 'mean_monthly_demand', 'std': 'std_monthly_demand'}, inplace=True)

# Calcola la media quantità per ordine per cliente (media dei quantity)
mean_quantity_per_order = delivery_data.groupby('location_id')['quantity'].mean().reset_index()
mean_quantity_per_order.rename(columns={'quantity': 'mean_quantity_per_order'}, inplace=True)

# Unisci tutti i dati
top_20_data = top_20_total_qty.merge(stats_monthly, on='location_id', how='left')
top_20_data = top_20_data.merge(mean_quantity_per_order, on='location_id', how='left')

# Ordina per quantità totale
top_20_data = top_20_data.sort_values('total_quantity', ascending=False)

fig, ax1 = plt.subplots(figsize=(14,7))

bars = ax1.bar(top_20_data['location_id'].astype(str), top_20_data['total_quantity'], color='skyblue', edgecolor='black')
ax1.set_xlabel('Clienti (ID)')
ax1.set_ylabel('Quantità totale ordinata', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticks(range(len(top_20_data)))
ax1.set_xticklabels(top_20_data['location_id'].astype(str), rotation=45)

# Etichetta sopra ogni barra per domanda totale
for bar, qty in zip(bars, top_20_data['total_quantity']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width() / 2, height + 0.05 * height, f"{int(qty)}", ha='center', va='bottom', fontsize=9, color='black')

# Numero intero della media quantità per ordine, posizionato poco sopra l'asse x
y_pos = +0.005 * max(top_20_data['total_quantity'])
for i, mean_order in enumerate(top_20_data['mean_quantity_per_order']):
    ax1.text(i, y_pos, f"{int(round(mean_order))}", ha='center', va='bottom', fontsize=10, color='blue')

ax2 = ax1.twinx()
ln1 = ax2.plot(top_20_data['location_id'].astype(str), top_20_data['std_monthly_demand'], 'ro-', label='Deviazione Standard')
ax2.set_ylabel('Deviazione standard mensile', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Aggiungi etichette sopra ogni punto della deviazione standard
for i, std in enumerate(top_20_data['std_monthly_demand']):
    ax2.text(i, std + 0.005 * max(top_20_data['std_monthly_demand']), f"{std:.1f}", color='red', ha='center', va='bottom', fontsize=9)

# Punto blu fittizio per legenda (media quantità per ordine)
ln2 = ax2.plot([], [], 'bo', label='Media quantità per ordine')

ax2.legend(handles=[ln1[0], ln2[0]], labels=['Deviazione Standard', 'Media quantità per ordine'], loc='upper right')

plt.title("Top 20 clienti per quantità totale ordinata - Media domanda per ordine e deviazione")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filtro ordini veri (is_event == 1)
orders_only = delivery_data[delivery_data['is_event'] == 1]
#count n ordini per cliente
top_20_num_orders = orders_only.groupby('location_id').size().reset_index(name='num_orders').sort_values('num_orders', ascending=False).head(20)    
# Calcola numero ordini mensili per cliente (solo ordini validi is_event=1)
monthly_orders = orders_only.copy()
monthly_orders['year_month'] = pd.to_datetime(monthly_orders['delivery_date']).dt.to_period('M')  # conversione sicura

monthly_num_orders = monthly_orders.groupby(['location_id', 'year_month']).size().reset_index(name='num_orders_per_month')

# Calcola media e deviazione standard del numero di ordini mensili per i top 20 clienti
stats_monthly = monthly_num_orders[monthly_num_orders['location_id'].isin(top_20_num_orders['location_id'])].groupby('location_id')['num_orders_per_month'].agg(['mean', 'std']).reset_index()
stats_monthly.rename(columns={'mean': 'mean_monthly_orders', 'std': 'std_monthly_orders'}, inplace=True)

# Unisci tutto
top_20_data = top_20_num_orders.merge(stats_monthly, on='location_id', how='left')
top_20_data = top_20_data.merge(mean_quantity_per_order, on='location_id', how='left')

# Resto del plot rimane invariato, aggiorna solo le colonne usate per il plot come segue:

fig, ax1 = plt.subplots(figsize=(14,7))

bars = ax1.bar(top_20_data['location_id'].astype(str), top_20_data['num_orders'], color='skyblue', edgecolor='black')
ax1.set_xlabel('Clienti (ID)')
ax1.set_ylabel('Numero totale di ordini', color='skyblue')
ax1.tick_params(axis='y', labelcolor='skyblue')
ax1.set_xticks(range(len(top_20_data)))
ax1.set_xticklabels(top_20_data['location_id'].astype(str), rotation=45)

for bar, n_ord in zip(bars, top_20_data['num_orders']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width() / 2, height + 0.05 * height, f"{int(n_ord)}", ha='center', va='bottom', fontsize=9, color='black')

y_pos = +0.005 * max(top_20_data['num_orders'])
for i, mean_order in enumerate(top_20_data['mean_monthly_orders']):
    ax1.text(i, y_pos, f"{mean_order:.1f}", ha='center', va='bottom', fontsize=10, color='blue')

ax2 = ax1.twinx()
ln1 = ax2.plot(top_20_data['location_id'].astype(str), top_20_data['std_monthly_orders'], 'ro-', label='Deviazione Standard Ordini Mensili')
ax2.set_ylabel('Deviazione standard mensile ordini', color='red')
ax2.tick_params(axis='y', labelcolor='red')

for i, std in enumerate(top_20_data['std_monthly_orders']):
    ax2.text(i, std + 0.05 * max(top_20_data['std_monthly_orders']), f"{std:.1f}", color='red', ha='center', va='bottom', fontsize=9)

ln2 = ax2.plot([], [], 'bo', label='Media ordini mensili')

ax2.legend(handles=[ln1[0], ln2[0]], labels=['Deviazione Standard', 'Media mensile ordini'], loc='upper right')

plt.title("Top 20 clienti per numero di ordini (is_event=1) - Media e deviazione mensile ordini")
plt.tight_layout()
plt.show()



# Clustering con divsione in rettangolo uguali

In [ ]:
# imports
import import_ipynb
import performance_calc as pc

In [ ]:


def cluster_rettangoli(delivery_points, verbose = False):

    # Calcola min e max per lat e lon
    lat_min, lat_max = delivery_points['lat'].min(), delivery_points['lat'].max()
    lon_min, lon_max = delivery_points['lon'].min(), delivery_points['lon'].max()

    lat_step = (lat_max - lat_min) / 8
    lon_step = (lon_max - lon_min) / 8

    def assign_cluster(row):
        lat_cluster = min(int((row['lat'] - lat_min) / lat_step), 7)
        lon_cluster = min(int((row['lon'] - lon_min) / lon_step), 7)
        return (lat_cluster, lon_cluster)

    
    delivery_points['cluster'] = delivery_points.apply(assign_cluster, axis=1)

    clusters = delivery_points.groupby('cluster')['location_id'].apply(list).to_dict()

    if verbose:
        for cluster, loc_ids in clusters.items():
            print(f"Cluster {cluster}:")
            print(loc_ids)
            print()
    print("Numero di cluster con almeno un punto:", delivery_points['cluster'].nunique())

    return delivery_points, clusters    



### Graficare clusters

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dati del clustering
clusters_unique = clusters_df['cluster'].unique()
n_clusters = len(clusters_unique)
colors = plt.cm.get_cmap('tab20', n_clusters)

# Calcolo dei limiti
lat_min, lat_max = clusters_df['lat'].min(), clusters_df['lat'].max()
lon_min, lon_max = clusters_df['lon'].min(), clusters_df['lon'].max()

# Step di divisione in 8 parti
lat_steps = np.linspace(lat_min, lat_max, 9)
lon_steps = np.linspace(lon_min, lon_max, 9)

plt.figure(figsize=(12,8))

# Disegna la griglia 8x8
for lat in lat_steps:
    plt.axhline(lat, color='gray', linestyle='--', linewidth=0.7)
for lon in lon_steps:
    plt.axvline(lon, color='gray', linestyle='--', linewidth=0.7)

# Scatter plot per ogni cluster
for i, cluster_id in enumerate(clusters_unique):
    cluster_points = clusters_df[clusters_df['cluster'] == cluster_id]
    lat = cluster_points['lat']
    lon = cluster_points['lon']
    plt.scatter(lon, lat, label=f'Cluster {cluster_id}', color=colors(i), s=30)

plt.xlabel('Longitudine')
plt.ylabel('Latitudine')
plt.title('Cluster geografici divisi 8x8 con griglia')
plt.legend(title='Cluster')
plt.grid(False)
plt.show()


# clustering con rettangoli logaritmici - alta densità verso il centro

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Esempio: dati clustering_data con colonne ['location_id', 'lat', 'lon']

def cluster_rettangoli_log(delivery_points):

    # Bounding box
    lat_min, lat_max = delivery_points['lat'].min(), delivery_points['lat'].max()
    lon_min, lon_max = delivery_points['lon'].min(), delivery_points['lon'].max()

    n = 8  # dimensione griglia

    # Crea peso simmetrico per dimensione celle, più piccolo al centro, normalizzato
    half = n // 2
    base = np.linspace(1, 3, half)  # cresce da 1 a 3 verso i bordi
    weights_1d = np.concatenate([base[::-1], base]) if n % 2 == 0 else np.concatenate([base[::-1], [max(base)], base])
    weights_1d /= np.sum(weights_1d)

    # Genera i bordi delle celle cumulando pesi
    lat_edges = lat_min + np.insert(np.cumsum(weights_1d) * (lat_max - lat_min), 0, 0)
    lon_edges = lon_min + np.insert(np.cumsum(weights_1d) * (lon_max - lon_min), 0, 0)

    # Funzione per determinare in quale cella cade un punto
    def find_cell(coord, edges):
        for i in range(len(edges)-1):
            if edges[i] <= coord < edges[i+1]:
                return i
        return len(edges)-2  # se il punto è esattamente all'ultimo bordo

    # Assegna cluster (riga, colonna)
    def assign_cluster(row):
        lat_idx = find_cell(row['lat'], lat_edges)
        lon_idx = find_cell(row['lon'], lon_edges)
        return (lat_idx, lon_idx)

    delivery_points['cluster'] = delivery_points.apply(assign_cluster, axis=1)

    # Raggruppa location_id per cluster
    clusters = delivery_points.groupby('cluster')['location_id'].apply(list).to_dict()

    # Stampa cluster e clienti
    for cluster, locs in clusters.items():
        print(f"Cluster {cluster} ({len(locs)} clienti): {locs}")

    return delivery_points, clusters

clusters_df, clusters_dict = cluster_rettangoli_log(pc.delivery_points)


### Graficare clusters

In [ ]:
def plot_cluster(clusters_df): 
    # Plot
    fig, ax = plt.subplots(figsize=(12,10))

    # Bounding box
    lat_min, lat_max = clusters_df['lat'].min(), clusters_df['lat'].max()
    lon_min, lon_max = clusters_df['lon'].min(), clusters_df['lon'].max()

    n = 8  # dimensione griglia

    # Crea peso simmetrico per dimensione celle, più piccolo al centro, normalizzato
    half = n // 2
    base = np.linspace(1, 3, half)  # cresce da 1 a 3 verso i bordi
    weights_1d = np.concatenate([base[::-1], base]) if n % 2 == 0 else np.concatenate([base[::-1], [max(base)], base])
    weights_1d /= np.sum(weights_1d)

    # Genera i bordi delle celle cumulando pesi
    lat_edges = lat_min + np.insert(np.cumsum(weights_1d) * (lat_max - lat_min), 0, 0)
    lon_edges = lon_min + np.insert(np.cumsum(weights_1d) * (lon_max - lon_min), 0, 0)

    # Disegna i rettangoli
    for i in range(n):
        for j in range(n):
            lat_low = lat_edges[i]
            lat_high = lat_edges[i+1]
            lon_low = lon_edges[j]
            lon_high = lon_edges[j+1]
            rect = plt.Rectangle((lon_low, lat_low), lon_high - lon_low, lat_high - lat_low,
                             edgecolor='gray', facecolor='none', lw=1)
            ax.add_patch(rect)


    unique_clusters = clusters_df['cluster'].unique()
    import random

    random.seed(42)  # Per ripetibilità
    def random_color():
        return (random.random(), random.random(), random.random())

    cluster_color_map = {cluster: random_color() for cluster in unique_clusters}

    # Scatter punti colorati per cluster
    for cluster, points in clusters_df.groupby('cluster'):
        ax.scatter(points['lon'], points['lat'], label=f'Cluster {cluster}', color=cluster_color_map[cluster], s=30)

    plt.xlabel('Longitudine')
    plt.ylabel('Latitudine')
    plt.title('Cluster con celle dimensionate logaritmicamente')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(False)
    plt.show()

    print("Numero di cluster con almeno un punto:", clusters_df['cluster'].nunique())


### Run & routing

In [ ]:
import pickle

clusters_df, clusters_dict = cluster_rettangoli(pc.delivery_points_AS)
complete_list = []
# creating the list of list to exec routing
for list_of_ids in clusters_dict.values():
    complete_list.append(list_of_ids)

pc.calc_clusters_stats_AS(complete_list).to_csv('clustering_methods_performances/rettangoli_semplici_AS.csv')

# salva in pickle il file cluster_dict
with open('cluster_dicts/cluster_dict_rettangoli_semplici_AS.pkl', 'wb') as f:
    pickle.dump(clusters_dict, f)


# Now for ON
clusters_df, clusters_dict = cluster_rettangoli(pc.delivery_points_ON)
complete_list = []
# creating the list of list to exec routing
for list_of_ids in clusters_dict.values():
    complete_list.append(list_of_ids)

pc.calc_clusters_stats_ON(complete_list).to_csv('clustering_methods_performances/rettangoli_semplici_ON.csv')

# salva in pickle il file cluster_dict
with open('cluster_dicts/cluster_dict_rettangoli_semplici_ON.pkl', 'wb') as f:
    pickle.dump(clusters_dict, f)

In [ ]:

clusters_df, clusters_dict = cluster_rettangoli_log(pc.delivery_points_AS)
complete_list = []
# creating the list of list to exec routing
for list_of_ids in clusters_dict.values():
    complete_list.append(list_of_ids)

pc.calc_clusters_stats_AS(complete_list).to_csv('clustering_methods_performances/rettangoli_logaritmici_AS.csv')

# salva in pickle il file cluster_dict
with open('cluster_dicts/cluster_dict_rettangoli_logaritmici_AS.pkl', 'wb') as f:
    pickle.dump(clusters_dict, f)


# Now for ON
clusters_df, clusters_dict = cluster_rettangoli_log(pc.delivery_points_ON)
complete_list = []
# creating the list of list to exec routing
for list_of_ids in clusters_dict.values():
    complete_list.append(list_of_ids)

pc.calc_clusters_stats_ON(complete_list).to_csv('clustering_methods_performances/rettangoli_logaritmici_ON.csv')

# salva in pickle il file cluster_dict
with open('cluster_dicts/cluster_dict_rettangoli_logaritmici_ON.pkl', 'wb') as f:
    pickle.dump(clusters_dict, f)